In [63]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#         cv2.imshow('img',img)
#         cv2.waitKey(500)

cv2.destroyAllWindows()

In [64]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [326]:
input_img = cv2.imread('./test_images/test2.jpg')
input_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(input_img)

In [327]:
undist = cv2.undistort(input_img, mtx, dist, None, mtx)
plt.imshow(undist)

In [328]:
src = np.float32([[150,720],[590,450],[700,450],[1250,720]])
dst = np.float32([[200,720],[200,0],[980,0],[980,720]])
M = cv2.getPerspectiveTransform(src,dst)
corrected = cv2.warpPerspective(undist,M,(undist.shape[1], undist.shape[0]))
plt.imshow(corrected)
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [341]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return sxbinary

In [290]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
#         cv2.rectangle(out_img,(win_xleft_low,win_y_low),
#         (win_xleft_high,win_y_high),(0,255,0), 2) 
#         cv2.rectangle(out_img,(win_xright_low,win_y_low),
#         (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy>=win_y_low) & (nonzeroy<win_y_high) & (nonzerox>=win_xleft_low) & (nonzerox<win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy>=win_y_low) & (nonzeroy<win_y_high) & (nonzerox>=win_xright_low) & (nonzerox<win_xright_high)).nonzero()[0]
        # print(((nonzeroy>=win_y_low) & (nonzeroy<win_y_high) & (nonzerox>=win_xleft_low) & (nonzerox<win_xleft_high)))
        # Append these indices to the lishigh
        right_lane_inds.append(good_right_inds)
        left_lane_inds.append(good_left_inds)
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds)>minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds)>minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        pass # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###


    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fit = np.polyfit(lefty,leftx,2)
        right_fit = np.polyfit(righty,rightx,2)
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    zeros = np.array(np.where(binary_warped.T == 0))
    zeros_x = np.array(zeros[0])
    zeros_y = np.array(zeros[1])
    lane_region = ((left_fit[0]*zeros_y**2 + left_fit[1]*zeros_y + left_fit[2]<zeros_x)&(right_fit[0]*zeros_y**2 + right_fit[1]*zeros_y + right_fit[2]>zeros_x)).nonzero()[0]
    region = []
    region.append(lane_region)
    out_img[zeros_y[np.concatenate(region)],zeros_x[np.concatenate(region)]] = [0,255,0]
    
    curve_rad = measure_curvature_real(ploty,left_fit,right_fit)
    
    y_vehicle = binary_warped.shape[0]
    vehicle_offset = xm_per_pix*(left_fit[0]*y_vehicle**2 + left_fit[1]*y_vehicle + left_fit[2] + right_fit[0]*y_vehicle**2 + right_fit[1]*y_vehicle + right_fit[2] - binary_warped.shape[1])/2
    return out_img, curve_rad, vehicle_offset

In [280]:
def measure_curvature_real(ploty, left_fit_cr, right_fit_cr):  
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    curve_rad = int(min(left_curverad, right_curverad))

    return curve_rad

In [281]:
def addText(img,curve_rad,vehicle_offset):
    cv2.putText(img,'Radius of Curvature = '+str(curve_rad)+'(m)', (120,50),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
    if vehicle_offset>=0:
        cv2.putText(img,'Vehicle is %.2f'%vehicle_offset+'m left of center', (120,110),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
    else:
        cv2.putText(img,'Vehicle is %.2f'%-vehicle_offset+'m right of center', (120,110),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2) 
    return img

In [329]:
thresh = pipeline(corrected, s_thresh=(100, 255), sx_thresh=(20, 100))
lane_region, curve_rad, vehicle_offset = fit_polynomial(thresh)
M_inv = cv2.getPerspectiveTransform(dst,src)
lanes_unwarped = cv2.warpPerspective(lane_region,M_inv,(undist.shape[1], undist.shape[0]))
annotated_lanes = cv2.addWeighted(undist,0.8,lanes_unwarped,0.2,1)
result = addText(annotated_lanes,curve_rad, vehicle_offset)
plt.imshow(result)

In [283]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [343]:
def process_image(frame):
    undist = cv2.undistort(frame, mtx, dist, None, mtx)
    corrected = cv2.warpPerspective(undist,M,(undist.shape[1], undist.shape[0]))
    thresh = pipeline(corrected, s_thresh=(80, 255), sx_thresh=(20, 100))
    lane_region, curve_rad, vehicle_offset = fit_polynomial(thresh)
    M_inv = cv2.getPerspectiveTransform(dst,src)
    lanes_unwarped = cv2.warpPerspective(lane_region,M_inv,(undist.shape[1], undist.shape[0]))
    annotated_lanes = cv2.addWeighted(undist,0.5,lanes_unwarped,0.5,1)
    result = addText(annotated_lanes,curve_rad, vehicle_offset)
    return result

In [348]:
white_output = 'challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("challenge_video.mp4")
# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

                                                                                                                       
t:  49%|███████████████████████████████▏                                | 615/1260 [4:52:14<02:40,  4.01it/s, now=None]
                                                                                                                       
t:  49%|███████████████████████████████▏                                | 615/1260 [4:24:45<02:27,  4.38it/s, now=None]

t:   0%|                                                                             | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video challenge_video_output.mp4.
Moviepy - Writing video challenge_video_output.mp4





t:   0%|▎                                                                    | 2/485 [00:00<00:49,  9.73it/s, now=None]

t:   1%|▍                                                                    | 3/485 [00:00<01:06,  7.28it/s, now=None]

t:   1%|▌                                                                    | 4/485 [00:00<01:17,  6.21it/s, now=None]

t:   1%|▋                                                                    | 5/485 [00:00<01:20,  5.95it/s, now=None]

t:   1%|▊                                                                    | 6/485 [00:01<01:24,  5.64it/s, now=None]

t:   1%|▉                                                                    | 7/485 [00:01<01:27,  5.44it/s, now=None]

t:   2%|█▏                                                                   | 8/485 [00:01<01:29,  5.32it/s, now=None]

t:   2%|█▎                                                                   | 9/485 [00:01<01:31,  5.19it/s, now=None]

t:   2%|█▍                    

t:  14%|█████████▎                                                          | 66/485 [00:14<01:37,  4.28it/s, now=None]

t:  14%|█████████▍                                                          | 67/485 [00:14<01:36,  4.33it/s, now=None]

t:  14%|█████████▌                                                          | 68/485 [00:14<01:35,  4.37it/s, now=None]

t:  14%|█████████▋                                                          | 69/485 [00:15<01:38,  4.22it/s, now=None]

t:  14%|█████████▊                                                          | 70/485 [00:15<01:35,  4.33it/s, now=None]

t:  15%|█████████▉                                                          | 71/485 [00:15<01:40,  4.13it/s, now=None]

t:  15%|██████████                                                          | 72/485 [00:15<01:38,  4.17it/s, now=None]

t:  15%|██████████▏                                                         | 73/485 [00:16<01:39,  4.12it/s, now=None]

t:  15%|██████████▍             

t:  27%|█████████████████▉                                                 | 130/485 [00:28<01:15,  4.68it/s, now=None]

t:  27%|██████████████████                                                 | 131/485 [00:28<01:14,  4.76it/s, now=None]

t:  27%|██████████████████▏                                                | 132/485 [00:28<01:13,  4.82it/s, now=None]

t:  27%|██████████████████▎                                                | 133/485 [00:28<01:14,  4.70it/s, now=None]

t:  28%|██████████████████▌                                                | 134/485 [00:29<01:14,  4.72it/s, now=None]

t:  28%|██████████████████▋                                                | 135/485 [00:29<01:14,  4.71it/s, now=None]

t:  28%|██████████████████▊                                                | 136/485 [00:29<01:13,  4.72it/s, now=None]

t:  28%|██████████████████▉                                                | 137/485 [00:29<01:16,  4.57it/s, now=None]

t:  28%|███████████████████     

...

t:  53%|███████████████████████████████████▋                               | 258/485 [00:57<00:52,  4.35it/s, now=None]

t:  53%|███████████████████████████████████▊                               | 259/485 [00:58<00:51,  4.39it/s, now=None]

t:  54%|███████████████████████████████████▉                               | 260/485 [00:58<00:51,  4.38it/s, now=None]

t:  54%|████████████████████████████████████                               | 261/485 [00:58<00:51,  4.36it/s, now=None]

t:  54%|████████████████████████████████████▏                              | 262/485 [00:58<00:51,  4.35it/s, now=None]

t:  54%|████████████████████████████████████▎                              | 263/485 [00:59<00:50,  4.40it/s, now=None]

t:  54%|████████████████████████████████████▍                              | 264/485 [00:59<00:49,  4.44it/s, now=None]

t:  55%|████████████████████████████████████▌                              | 265/485 [00:59<00:49,  4.40it/s, now=None]

t:  55%|████████████████████████

t:  66%|████████████████████████████████████████████▍                      | 322/485 [01:12<00:37,  4.34it/s, now=None]

t:  67%|████████████████████████████████████████████▌                      | 323/485 [01:12<00:37,  4.29it/s, now=None]

t:  67%|████████████████████████████████████████████▊                      | 324/485 [01:13<00:36,  4.39it/s, now=None]

t:  67%|████████████████████████████████████████████▉                      | 325/485 [01:13<00:36,  4.39it/s, now=None]

t:  67%|█████████████████████████████████████████████                      | 326/485 [01:13<00:35,  4.45it/s, now=None]

t:  67%|█████████████████████████████████████████████▏                     | 327/485 [01:13<00:35,  4.41it/s, now=None]

t:  68%|█████████████████████████████████████████████▎                     | 328/485 [01:14<00:35,  4.37it/s, now=None]

t:  68%|█████████████████████████████████████████████▍                     | 329/485 [01:14<00:35,  4.38it/s, now=None]

t:  68%|████████████████████████

t:  80%|█████████████████████████████████████████████████████▎             | 386/485 [01:27<00:23,  4.13it/s, now=None]

t:  80%|█████████████████████████████████████████████████████▍             | 387/485 [01:27<00:23,  4.14it/s, now=None]

t:  80%|█████████████████████████████████████████████████████▌             | 388/485 [01:27<00:22,  4.23it/s, now=None]

t:  80%|█████████████████████████████████████████████████████▋             | 389/485 [01:27<00:21,  4.41it/s, now=None]

t:  80%|█████████████████████████████████████████████████████▉             | 390/485 [01:28<00:21,  4.50it/s, now=None]

t:  81%|██████████████████████████████████████████████████████             | 391/485 [01:28<00:21,  4.42it/s, now=None]

t:  81%|██████████████████████████████████████████████████████▏            | 392/485 [01:28<00:22,  4.22it/s, now=None]

t:  81%|██████████████████████████████████████████████████████▎            | 393/485 [01:28<00:21,  4.26it/s, now=None]

t:  81%|████████████████████████

t:  93%|██████████████████████████████████████████████████████████████▏    | 450/485 [01:40<00:07,  4.85it/s, now=None]

t:  93%|██████████████████████████████████████████████████████████████▎    | 451/485 [01:40<00:06,  4.94it/s, now=None]

t:  93%|██████████████████████████████████████████████████████████████▍    | 452/485 [01:41<00:06,  4.83it/s, now=None]

t:  93%|██████████████████████████████████████████████████████████████▌    | 453/485 [01:41<00:06,  4.90it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████████▋    | 454/485 [01:41<00:06,  4.91it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████████▊    | 455/485 [01:41<00:06,  4.83it/s, now=None]

t:  94%|██████████████████████████████████████████████████████████████▉    | 456/485 [01:42<00:06,  4.81it/s, now=None]

t:  94%|███████████████████████████████████████████████████████████████▏   | 457/485 [01:42<00:05,  4.85it/s, now=None]

t:  94%|████████████████████████

Moviepy - Done !
Moviepy - video ready challenge_video_output.mp4
Wall time: 1min 49s


In [349]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [344]:
input_img = cv2.imread('./test_images/test5.jpg')
result = process_image(input_img)
plt.imshow(result)